 https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

In [6]:
import string

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# load document
in_filename = 'output1989.txt'
doc = load_doc(in_filename)
print(doc[:200])

# clean document
tokens = clean_doc(doc)
tokens = [''.join([i for i in x if i.isalpha()]) for x in tokens if x != '$$$']
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'output1989_clean.txt'
save_doc(sequences, out_filename)

Chunking in a Connectionist Network
$$$
David S. Touretzky, Carnegie Mellon University
$$$
TOURETZKY Change Butter e 0 MOT °0 °o DEL O —o INS °0 5 4 
3 2 mm "E" 1 Modul* Input Butler i A B C D 
Figure
['chunking', 'in', 'a', 'connectionist', 'network', 'david', 's', 'touretzky', 'carnegie', 'mellon', 'university', 'touretzky', 'change', 'butter', 'e', 'mot', 'del', 'o', 'ins', 'mm', 'e', 'modul', 'input', 'butler', 'i', 'a', 'b', 'c', 'd', 'figure', 'part', 'of', 'a', 'connectionist', 'network', 'for', 'applying', 'rewrite', 'mles', 'to', 'strings', 'the', 'symbols', 'from', 'which', 'strings', 'are', 'composed', 'are', 'binary', 'feature', 'vectors', 'the', 'experiment', 'reported', 'here', 'uses', 'a', 'representanon', 'with', 'five', 'phonetic', 'features', 'organized', 'as', 'one', 'group', 'of', 'two', 'features', 'and', 'one', 'of', 'three', 'features', 'in', 'real', 'phonology', 'there', 'are', 'many', 'more', 'features', 'they', 'encode', 'the', 'place', 'and', 'manner', 'of', 

In [1]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = 'output1989_clean.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
lines = lines[:5000]

Using TensorFlow backend.


In [ ]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(128, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=200)

# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 50)            59100     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 128)           91648     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 1182)              152478    
Total params: 451,322
Trainable params: 451,322
Non-trainable params: 0
_________________________________________________________________
None

Epoch 1/200
5000/5000 [==============================] - 21s 4ms/step - loss: 6.4578 - accuracy: 0.0674
Epoc

5000/5000 [==============================] - 16s 3ms/step - loss: 3.0414 - accuracy: 0.2648
Epoch 68/200
5000/5000 [==============================] - 17s 3ms/step - loss: 2.9738 - accuracy: 0.2822
Epoch 69/200
5000/5000 [==============================] - 17s 3ms/step - loss: 2.9339 - accuracy: 0.2814
Epoch 70/200
5000/5000 [==============================] - 16s 3ms/step - loss: 2.8912 - accuracy: 0.3012
Epoch 71/200
5000/5000 [==============================] - 18s 4ms/step - loss: 2.8711 - accuracy: 0.3016
Epoch 72/200
5000/5000 [==============================] - 15s 3ms/step - loss: 2.8004 - accuracy: 0.3130
Epoch 73/200
5000/5000 [==============================] - 18s 4ms/step - loss: 2.7527 - accuracy: 0.3324
Epoch 74/200
5000/5000 [==============================] - 16s 3ms/step - loss: 2.7504 - accuracy: 0.3302
Epoch 75/200
5000/5000 [==============================] - 18s 4ms/step - loss: 2.7298 - accuracy: 0.3292
Epoch 76/200
5000/5000 [==============================] - 16s 3ms/st

5000/5000 [==============================] - 14s 3ms/step - loss: 0.8295 - accuracy: 0.8042
Epoch 145/200
5000/5000 [==============================] - 15s 3ms/step - loss: 0.7913 - accuracy: 0.8180
Epoch 146/200
5000/5000 [==============================] - 15s 3ms/step - loss: 0.8419 - accuracy: 0.8102
Epoch 147/200
5000/5000 [==============================] - 14s 3ms/step - loss: 0.8259 - accuracy: 0.8156
Epoch 148/200
5000/5000 [==============================] - 17s 3ms/step - loss: 0.7835 - accuracy: 0.8250
Epoch 149/200
5000/5000 [==============================] - 14s 3ms/step - loss: 0.7139 - accuracy: 0.8370
Epoch 150/200
5000/5000 [==============================] - 15s 3ms/step - loss: 0.6706 - accuracy: 0.8536
Epoch 151/200
5000/5000 [==============================] - 15s 3ms/step - loss: 0.6821 - accuracy: 0.8524
Epoch 152/200
5000/5000 [==============================] - 14s 3ms/step - loss: 0.6622 - accuracy: 0.8526
Epoch 153/200
5000/5000 [==============================] - 1

In [3]:
model.save('model.h5')

In [4]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'output1989_clean.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
lines = lines[:5000]
seq_length = len(lines[0].split()) - 1

# load the model
model = load_model('model.h5')

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

easier it can concentrate on learning just the policies of the speakers language this paper makes no assumption that policies require explicit symbolic representations in speakers heads rather it shows that chunking can occur even when there is no working memory trace available and new rules cannot be constructed symbolically the



In [5]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated)

connectionist chunker acquires its rules incrementally through selfsupervised backpropagation and rehearsal of prior knowledge further experiments are planned to analyze the representations the chunker develops acknowledgments tion grant and by the office of naval research under contract number i thank deirdre wheeler marco zagha michael witbrock and george lakoff for discussions that contributed to this work and gillette elvgren for help with the simulations references newell a the william james lectures unified theories of cognition given at harvard university laird j e newell a and rosenbloom p s soar an architecture for general intelligence artificial intelligence pinker s and prince
